# Intel® Neural Compressor Sample for Tensorflow

## Introduction

This is a demo to show an End-To-End pipeline to speed up AI model by Intel® Neural Compressor.

1. Train a CNN AlexNet model by Keras and Intel Optimization for Tensorflow based on dataset MNIST.

2. Quantize the frozen PB model file by Intel® Neural Compressor to INT8 model.

3. Test and compare the performance of FP32 and INT8 model by same script.


## Code
Please refer to [README.md](README.md).

## Check Script

In [ ]:
!cat run_inc_ft_mnist_sample.sh

## Prepare Running Environment

Please refer to [README.md](README.md).

## Run in Intel® DevCloud

Job submit to compute node with the property 'clx' or 'icx' or 'spr' which support Intel® Deep Learning Boost (avx512_vnni).

In [ ]:
!qsub run_inc_ft_mnist_sample.sh -d `pwd` -l nodes=1:icx:ppn=2

Check job status

In [ ]:
!qstat

## Check Result

### Check Result in Log File

In [ ]:
!tail -23 run_inc_ft_mnist_sample.sh.o1842253

Check Result in PNG file

In [ ]:
from IPython.display import Image, display

listOfImageNames = ['fp32_int8_aboslute.png',
                    'fp32_int8_times.png']

for imageName in listOfImageNames:
    display(Image(filename=imageName))